In [1]:
#Проект чат-бот который считает % проверенных домашних заданий педагогами и отправляет автоматическое сообщение педагогу, если % проверки ниже 75%

In [ ]:
#Библеотека для работы с чат-ботом телеграм
import telebot
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import threading
import time
from datetime import datetime

# Установите свои учетные данные для Google Sheets
SCOPE = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

# Замените на путь к вашему файлу credentials.json
CREDENTIALS_FILE = "C:/Users/1/Desktop/Project (Bot)/projectbot-447014-437b4c89d4a6.json"
credentials = ServiceAccountCredentials.from_json_keyfile_name(CREDENTIALS_FILE, SCOPE)

# Подключение к Google Sheets
client = gspread.authorize(credentials)
spreadsheet_name = "Отчёт по дз"
sheet = client.open(spreadsheet_name).sheet1

# Создание бота
bot = telebot.TeleBot('7809768765:AAHOm3QIEj-TEE-XTWZ9_Y8R0Tn0rzK8hwk')

# Ваши Chat ID
MY_CHAT_ID = 699343727  # Ваш Chat ID
MK_CHAT_ID = 366888178  # Другой Chat ID
JULIACHERNOVA_CHAT_ID = 1831617416  # Ещё один Chat ID

def calculate_report():
    try:
        # Получение данных из Google Sheets
        data = sheet.get_all_values()
        headers = data[0]  # Заголовки таблицы
        rows = data[1:]    # Данные таблицы

        # Создание DataFrame из данных таблицы
        df = pd.DataFrame(rows, columns=headers)

        # Преобразование числовых данных в int
        df["Выдано"] = pd.to_numeric(df["Выдано"], errors="coerce")
        df["Проверено"] = pd.to_numeric(df["Проверено"], errors="coerce")

        # Вычисление процента проверки
        df["Процент проверки"] = (df["Проверено"] / df["Выдано"] * 100).fillna(0).astype(int)

        # Формирование отчёта
        report_lines = ["*Отчёт по проверенным заданиям*:"]

        for _, row in df.iterrows():
            if row['Процент проверки'] == 0:
                report_lines.append(
                    f"*{row['ФИО преподавателя']}*\n"
                    f"Процент проверки: Нет выданных заданий\n"
                )
            else:
                report_lines.append(
                    f"*{row['ФИО преподавателя']}*\n"
                    f"Процент проверки: {row['Процент проверки']}%\n"
                )

        report_message = "\n\n".join(report_lines)
        return report_message
    except Exception as e:
        return f"Ошибка при формировании отчёта: {e}"

def send_scheduled_message():
    target_time = "17:26"  # Замените на нужное время (часы:минуты)
    while True:
        # Текущее время
        now = datetime.now().strftime("%H:%M")
        if now == target_time:
            try:
                # Формирование отчёта
                report_message = calculate_report()
                
                # Отправка отчёта в Telegram
                bot.send_message(MY_CHAT_ID, report_message, parse_mode="Markdown")
                bot.send_message(MK_CHAT_ID, report_message, parse_mode="Markdown")
                bot.send_message(JULIACHERNOVA_CHAT_ID, report_message, parse_mode="Markdown")
                print(f"[{datetime.now()}] Сообщения отправлены!")
            except Exception as e:
                print(f"[{datetime.now()}] Ошибка при отправке сообщения: {e}")
            
            # Ждём 60 секунд, чтобы избежать повторной отправки в течение той же минуты
            time.sleep(60)
        else:
            # Проверяем каждую секунду
            time.sleep(1)

# Создание и запуск потока
thread = threading.Thread(target=send_scheduled_message, daemon=True)
thread.start()

@bot.message_handler(commands=['start'])
def start(message):
    bot.send_message(message.chat.id, f'Привет, {message.from_user.first_name} {message.from_user.last_name}!')

@bot.message_handler(commands=['report'])
def report(message):
    bot.send_message(MY_CHAT_ID, f'Команду /report выполняет: {message.from_user.first_name} {message.from_user.last_name} {message.from_user.username} {message.from_user.id}')
    try:
        # Формирование и отправка отчёта
        report_message = calculate_report()
        bot.send_message(message.chat.id, report_message, parse_mode="Markdown")
    except Exception as e:
        bot.send_message(message.chat.id, f"Ошибка при чтении данных: {e}")

# Запуск бота
print("Бот запущен. Поток для автоматической отправки сообщений работает...")
bot.polling(none_stop=True)


Бот запущен. Поток для автоматической отправки сообщений работает...
